#Deep Learning Object Detection
## This Notebook contains
- Download/Refresh of neccessary Libraries and pretrained models
- Protobuf Compiler installation for the runtime
- Training of pretrained models (Transfer-Learning)
- Creation of the Inference Graph (neural Network, not for edgeTPU)
- Creation of a tensorflow lite (.tflite) model and compilation for usage in Edge-TPU 
- Running the IG on images from "testimages" folder
- creating a video from "Saved_detections" folder 

##Before you start 
make sure you did the following:
0.   use Prepare_new_Training.ipynb to create a fresh folder structure and files for a new training session
1.   Upload Training-Images into "JPEGImages" folder
2.   Upload Training-Annotations into "Annotations" folder (Create Annotations with LabelImg.py)
3.   Clear the CP-Folder (else would result in errors)
4.   Do not change the name of the notebook. Only the last digit can be changed. (Has to do with Prepare_new_Training.ipynb)

Pro-Tip: Keep Colab running
from: https://medium.com/@shivamrawat_756/how-to-prevent-google-colab-from-disconnecting-717b88a128c0
Press Ctrl+Shift+i

function ClickConnect(){
 console.log(“Working….”); 
 document.querySelector(“paper-button#comments”).click()
}
setInterval(ClickConnect,60000)


# Mounting Google Drive

In [3]:
#only if cloud computing
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


##(Optional) Refresh the latest "models" directory from tensorflow into DeepLearning-Folder
##Careful: Never change a running system. better back up the models dictionary befor you replace it.
Source: https://github.com/tensorflow/models

In [ ]:
%cd /content/gdrive/My Drive/DeepLearning
!git clone https://github.com/tensorflow/models.git

##(Optional) Refresh pre trained models in DeepLearning-Folder
A nice spot to find pre-trained models from Tensorflow is here: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md

In [ ]:
%cd /content/gdrive/My Drive/DeepLearning
!pip3 install wget
import os
root_path=os.getcwd()
print(root_path)
# define the name of the directory to be created
folders_to_create = [root_path+"/pre_trained_models"]
for path in folders_to_create:
  try:  
      os.makedirs(path)
  except OSError:  
      print ("Creation of the directory %s failed" % path)
  else:  
      print ("Successfully created the directory %s" % path)
import wget
print('Beginning file download with wget module')
urls = ['faster_rcnn_resnet101_coco_2018_01_28.tar.gz',
        'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz',
        'ssd_mobilenet_v2_coco_2018_03_29.tar.gz',
        'ssd_mobilenet_v3_large_coco_2019_08_14.tar.gz']
for url in urls:
  wget.download("http://download.tensorflow.org/models/object_detection/"+url, root_path+"/pre_trained_models/"+url)
  #!wget -P %pwd/pre_trained_models http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_2018_01_28.tar.gz
  import tarfile
  fname=root_path+"/pre_trained_models/"+url
  if (fname.endswith("tar.gz")):
      tar = tarfile.open(fname, "r:gz")
      tar.extractall(path=root_path+"/pre_trained_models")
      tar.close()
  elif (fname.endswith("tar")):
      tar = tarfile.open(fname, "r:")
      tar.extractall(path=root_path+"/pre_trained_models")
      tar.close()
  print("download and unpacking successfull: ",url)

#!git clone https://github.com/tensorflow/models.git 

#Environment Setup

##(Required for each session) Set Python path and create protoc files
therefore create protoc files in "protos" folder of "models" and set Paths

In [ ]:
%set_env PYTHONPATH=/content/gdrive/My Drive/DeepLearning/models/research:/content/gdrive/My Drive/DeepLearning/models/research/slim
# %set_env PYTHONPATH=/content/gdrive/My Drive/DeepLearning/models/research/object_detection
!apt-get install protobuf-compiler python-pil python-lxml python-tk

%cd /content/gdrive/My Drive/DeepLearning/models/research
%ls
#%set_env PYTHONPATH=/content/gdrive/My Drive/DeepLearning/models/research:/content/gdrive/My Drive/DeepLearning/models/research/slim
!protoc object_detection/protos/*.proto --python_out=.
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
#If unsure if protoc work correctly run model builder test script
#!python object_detection/builders/model_builder_test.py

env: PYTHONPATH=/content/gdrive/My Drive/DeepLearning/models/research:/content/gdrive/My Drive/DeepLearning/models/research/slim
Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 11 not upgraded.
Need to get 1,791 kB of archives.
After this operation, 7,807 kB of additional disk space will be used.
Get:1 http://ar

##Run the Model builder test script

In [ ]:
#If unsure if protoc work correctly run model builder test script
!pip install tf_slim
# %tensorflow_version 1.x
!python object_detection/builders/model_builder_test.py

     |████████████████████████████████| 358kB 7.0MB/s 
2020-11-06 11:30:18.163671: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


##Adapt Paths of .config-files to desired Trainng session Folder (FasterRCNN_TV(X))

Adding a new pre trained model:
the config files of pre trained models can be downloaded from here: https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs
It is neccessary to adapt the paths of .pbtxt, .model and .ckpt like done in jonen_ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.config.
###Note:
Some Config-Files of pre-trained models have a too large batch size or have not specified the layers we want to freeze in the model (see more info here: https://coral.withgoogle.com/docs/edgetpu/retrain-detection/#select-your-model)

In [ ]:
%cd /content/gdrive/My Drive/DeepLearning
import os
print(os.getcwd())
to_="FasterRCNN_TV6" #Foldername where CP,IG and Dataset Folder are located
#where should the Training folder be located?
root_path=os.getcwd() # 
#root_path_windows=root_path.replace("'\'",'/')
config_file_to=root_path+"/jonen_faster_rcnn_resnet101_coco_2018_01_28.config"
file=open(config_file_to, "r")
#print(file)
newfile=file.read()
newfile=newfile.replace("MODEL_PATH_TO_BE_CONFIGURED/",root_path+"/pre_trained_models/faster_rcnn_resnet101_coco_2018_01_28/")
newfile=newfile.replace("PATH_TO_BE_CONFIGURED/",root_path+"/"+to_+"/Tensorflow_API-Custom_object_detection/dataset/")
file.close()
file=open(root_path+"/pre_trained_models/faster_rcnn_resnet101_coco_2018_01_28/"+"pipeline.config", "w")
file.write(newfile)
print("Replaced Paths in File#####\n")
print(newfile)
file.close()

config_file_to=root_path+"/jonen_ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.config"
file=open(config_file_to, "r")
#print(file)
newfile=file.read()
newfile=newfile.replace("MODEL_PATH_TO_BE_CONFIGURED/",root_path+"/pre_trained_models/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/")
newfile=newfile.replace("PATH_TO_BE_CONFIGURED/",root_path+"/"+to_+"/Tensorflow_API-Custom_object_detection/dataset/")
file.close()
file=open(root_path+"/pre_trained_models/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/"+"pipeline.config", "w")
file.write(newfile)
print("Replaced Paths in File#####\n")
print(newfile)
file.close()

config_file_to=root_path+"/jonen_ssdlite_mobilenet_v3_large_320x320_coco.config"
file=open(config_file_to, "r")
#print(file)
newfile=file.read()
newfile=newfile.replace("MODEL_PATH_TO_BE_CONFIGURED/",root_path+"/pre_trained_models/ssdlite_mobilenet_v3_large_320x320_coco/")
newfile=newfile.replace("PATH_TO_BE_CONFIGURED/",root_path+"/"+to_+"/Tensorflow_API-Custom_object_detection/dataset/")
file.close()
file=open(root_path+"/pre_trained_models/ssdlite_mobilenet_v3_large_320x320_coco/"+"pipeline.config", "w")
file.write(newfile)
print("Replaced Paths in File#####\n")
print(newfile)
file.close()

config_file_to=root_path+"/jonen_ssd_mobilenet_v2_coco_2018_03_29.config"
file=open(config_file_to, "r")
#print(file)
newfile=file.read()
newfile=newfile.replace("MODEL_PATH_TO_BE_CONFIGURED/",root_path+"/pre_trained_models/ssd_mobilenet_v2_coco_2018_03_29/")
newfile=newfile.replace("PATH_TO_BE_CONFIGURED/",root_path+"/"+to_+"/Tensorflow_API-Custom_object_detection/dataset/")
file.close()
file=open(root_path+"/pre_trained_models/ssd_mobilenet_v2_coco_2018_03_29/"+"pipeline.config", "w")
file.write(newfile)
print("Replaced Paths in File#####\n")
print(newfile)
file.close()

config_file_to=root_path+"/jonen_ssd_mobilenet_v1_quantized_300x300_coco_2018_07_18.config"
file=open(config_file_to, "r")
#print(file)
newfile=file.read()
newfile=newfile.replace("MODEL_PATH_TO_BE_CONFIGURED/",root_path+"/pre_trained_models/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/")
newfile=newfile.replace("PATH_TO_BE_CONFIGURED/",root_path+"/"+to_+"/Tensorflow_API-Custom_object_detection/dataset/")
file.close()
file=open(root_path+"/pre_trained_models/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/"+"pipeline.config", "w")
file.write(newfile)
print("Replaced Paths in File#####\n")
print(newfile)
file.close()


print(">>>Maybe check if the paths are correct")

/content/gdrive/My Drive/DeepLearning
/content/gdrive/My Drive/DeepLearning
Replaced Paths in File#####

# Faster R-CNN with Resnet-50 (v1), configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  faster_rcnn {
    num_classes: 90
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_resnet50'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {

##Let's create a label.pbtxt file, which lists the labels from the different annotations e.g. "dog", "cat", "horse"

In [ ]:
%cd /content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/dataset
#PLEASE INSERT THE NAME(S) OF THE LABEL(S) INTO "labels" LIST
# labels=["Elevator_Panel","Elevator_Call","Elevator_Panel1","Call_Elevator2","Elevator_Panel3","Call_Elevator3"]
labels=["FC"]
#
file=open("label.pbtxt", "w")
i0=1
for label in labels:
  entry="item{\n\tid: %d\n\tname: '%s'\n}"%(i0,label)
  print(entry)
  file.write(entry)
  i0+=1
file.close()
print("successfully created file 'label.pbtxt'")

/content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/dataset
item{
	id: 1
	name: 'FC'
}
successfully created file 'label.pbtxt'


###Remove image and Annotation files which are doubled
(Execute Only Once)

In [ ]:
%cd /content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection
#at first clean up the annotations and images, delete all elements that do not
#have a corresponding image or annotation
import glob2
import os
#Images
count=0
protc_files=glob2.glob('./dataset/JPEGImages/*')
print("found Images: \n", len(protc_files))
images_names=os.listdir('./dataset/JPEGImages/')
images_names_pure=[name[0:-5] for name in images_names]
annotation_names=os.listdir('./dataset/Annotations/')
annotation_names_pure=[name[0:-4] for name in annotation_names]
for i2 in range(len(protc_files)):
  if images_names_pure[i2] not in annotation_names_pure:
    os.remove(protc_files[i2])
    # print('deleted file: ',protc_files[i2])
    count+=1
 #Annotations
protc_files=glob2.glob('./dataset/Annotations/*')
print("found Annotations: \n", len(protc_files))
print("%d images had no annotation. Removed them."%count)  
#     indic.append(i2)
# if len(indic)>0:
#   for i2 in range(len(indic)):
#     # os.remove(protc_files[indic[i2]])
#     print('deleted file: ',protc_files[indic[i2]])
# #Annotations
# indic=[]
# protc_files=glob2.glob('./dataset/Annotations/*')
# print("found Annotations: \n", len(protc_files))
# annotation_names=os.listdir('./dataset/Annotations/')
# annotation_names_pure=[name[0:-4] for name in annotation_names]
# for i2 in range(len(protc_files)):
#   if protc_files[i2].find("(")>-1: #-1 means no sign found
#     indic.append(i2)
# if len(indic)>0:
#   for i2 in range(len(indic)):
#     # os.remove(protc_files[indic[i2]])
#     print('deleted file: ',protc_files[indic[i2]])

      

/content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection
found Images: 
 600
found Annotations: 
 578
22 images had no annotation. Removed them.


# The Training


## (Once for new Training data) Convert VOC to tf record
Carefull: the folder which holds the annotation files (.xml) needs to have the same name as indicated in '< path >' in the .xml-files.

(Execute Only Once)

In [ ]:
# %tensorflow_version 1.x
!python /content/gdrive/My\ Drive/DeepLearning/jonen_create_pascal_tf_record.py \
--data_dir="/content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/dataset" \
--annotations_dir="/content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/dataset/Annotations" \
--output_path="/content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/dataset/train.record" \
--label_map_path="/content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/dataset/label.pbtxt"

2020-11-06 11:36:05.131075: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
I1106 11:36:06.593742 140716764567424 jonen_create_pascal_tf_record.py:156] On image 0 of 578
/content/gdrive/My Drive/DeepLearning/models/research/object_detection/utils/dataset_util.py:79: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not xml:
I1106 11:37:22.649760 140716764567424 jonen_create_pascal_tf_record.py:156] On image 100 of 578
I1106 11:38:38.754507 140716764567424 jonen_create_pascal_tf_record.py:156] On image 200 of 578
I1106 11:39:52.310090 140716764567424 jonen_create_pascal_tf_record.py:156] On image 300 of 578
I1106 11:41:05.574835 140716764567424 jonen_create_pascal_tf_record.py:156] On image 400 of 578
I1106 11:42:18.976651 140716764567424 jonen_create_pascal_tf_record.py:156] On image 500 of 578


##Train Model
Make sure dataset/CP is clear and there are no checkpoints stored

set the --pipeline of the desired pre-trained model
###Note
If you want to train a **quantized** Model for running in edge TPU make sure the delay in graph_rewriter (.config-file) is set to 1000 if you want to train 1200 Steps. Read more here https://medium.com/tensorflow/training-and-serving-a-realtime-mobile-object-detector-in-30-minutes-with-cloud-tpus-b78971cf1193

Sadly: MobileNet_V2_Quantized currently has Out-Of-Memory-Error, MobileNetV1 however works. (Nov, 2019)

In [9]:
!pip install tf_slim
""" COPY INTO Browser Console (Strg+Shift+i)
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button").click() 
}setInterval(ClickConnect,60000)

"""
#Version 2 with latest algorithms
# %cd /content/gdrive/My\ Drive/DeepLearning/models/research/
%cd /content/gdrive/My\ Drive/DeepLearning/models/research/
# !pip install tf_slim
%tensorflow_version 1.x
!python object_detection/model_main.py \
--pipeline_config_path="/content/gdrive/My Drive/DeepLearning/pre_trained_models/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/pipeline.config" \
--model_dir="/content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/CP" \
--sample_1_of_n_eval_examples=1 \
--alsologtostderr \
--num_train_steps=5000 \
#--pipeline_config_path="/content/gdrive/My Drive/DeepLearning/pre_trained_models/ssdlite_mobilenet_v3_large_320x320_coco/pipeline.config" \
#--pipeline_config_path="/content/gdrive/My Drive/DeepLearning/pre_trained_models/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/pipeline.config" \
#--pipeline_config_path="/content/gdrive/My Drive/DeepLearning/pre_trained_models/faster_rcnn_resnet101_coco_2018_01_28/pipeline.config" \
#--pipeline_config_path="/content/gdrive/My Drive/DeepLearning/pre_trained_models/ssd_mobilenet_v2_coco_2018_03_29/pipeline.config" \
#--pipeline_config_path="/content/gdrive/My Drive/DeepLearning/pre_trained_models/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/pipeline.config" \
#If MobileNetV2 is to be trained, please check if issue from https://github.com/tensorflow/models/issues/4066 is cleared after refreshing "models"

/content/gdrive/My Drive/DeepLearning/models/research
TensorFlow 1.x selected.
W1106 11:45:34.844598 140153470158720 model_lib.py:717] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 5000
I1106 11:45:34.845315 140153470158720 config_util.py:552] Maybe overwriting train_steps: 5000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1106 11:45:34.845834 140153470158720 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I1106 11:45:34.846028 140153470158720 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I1106 11:45:34.846212 140153470158720 config_util.py:552] Maybe overwriting eval_num_epochs: 1
INFO:tensorflow:Maybe overwriting load_pretrained: True
I1106 11:45:34.846503 140153470158720 config_util.py:552] Maybe overwriting load_pretrained: True
INFO:tensorflow:Ignoring config overr

At first train the model, finish when loss is below 0.1 with keyboard interrupt
below adapt the number of the checkpoint model you want to use --> model.ckpt-1691

##Option A: Save inference graph (no edge TPU)
Change number of model.ckpt-1780 (example), take checkpoint number from dataset/CP folder

In [ ]:
%cd /content/gdrive/My\ Drive/DeepLearning/
!python /content/gdrive/My\ Drive/DeepLearning/models/research/object_detection/export_inference_graph.py \
--input_type=image_tensor \
--pipeline_config_path="/content/gdrive/My Drive/DeepLearning/pre_trained_models/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/pipeline.config" \
--trained_checkpoint_prefix="/content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/CP/model.ckpt-844" \
--output_directory="/content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/IG"

#--pipeline_config_path="/content/gdrive/My Drive/DeepLearning/pre_trained_models/ssdlite_mobilenet_v3_large_320x320_coco/pipeline.config" \
#--pipeline_config_path="/content/gdrive/My Drive/DeepLearning/pre_trained_models/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/pipeline.config" \
#--pipeline_config_path="/content/gdrive/My Drive/DeepLearning/pre_trained_models/faster_rcnn_resnet101_coco_2018_01_28/pipeline.config" \
#--pipeline_config_path="/content/gdrive/My Drive/DeepLearning/pre_trained_models/ssd_mobilenet_v2_coco_2018_03_29/pipeline.config" \
#--pipeline_config_path="/content/gdrive/My Drive/DeepLearning/pre_trained_models/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/pipeline.config" \

##Option B: Save inference graph for Edge TPU
###Let's convert the checkpoint (model.ckpt-XXXX) to a .tflite model for usage in Edge devices (e.g. Raspberry pi and Coral Edge TPU)
Source: from https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/running_on_mobile_tensorflowlite.md found here https://stackoverflow.com/questions/54823475/tensorflow-object-detection-convert-pb-file-to-tflite

or also here: https://medium.com/tensorflow/training-and-serving-a-realtime-mobile-object-detector-in-30-minutes-with-cloud-tpus-b78971cf1193

In [10]:
#set the checkpoint number (trained_checkpoint_prefix)
# we must be in models/research
%cd /content/gdrive/My Drive/DeepLearning/models/research
!pip install tf_slim
%tensorflow_version 1.x
!python object_detection/export_tflite_ssd_graph.py \
--pipeline_config_path="/content/gdrive/My Drive/DeepLearning/pre_trained_models/ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18/pipeline.config" \
--trained_checkpoint_prefix="/content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/CP/model.ckpt-5000" \
--output_directory="/content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/IG" \
--add_postprocessing_op=true

/content/gdrive/My Drive/DeepLearning/models/research
Instructions for updating:
Please use `layer.__call__` method instead.
W1106 14:10:54.949431 140538822936448 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I1106 14:10:56.331570 140538822936448 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I1106 14:10:56.361295 140538822936448 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I1106 14:10:56.390420 140538822936448 convolutional_box_predictor.py:156] depth of additional conv before box pred

###Download tensorflow folder from git into the current environment (temporary)

In [11]:
#Problem here is that the paths from gdrive have spaces. 
#so we must load the tensorflow directory temporary into the current runtime
%cd /
%cd /content/
!git clone https://github.com/tensorflow/tensorflow.git

/
/content
Cloning into 'tensorflow'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 1031600 (delta 2), reused 6 (delta 2), pack-reused 1031590
Receiving objects: 100% (1031600/1031600), 615.16 MiB | 26.71 MiB/s, done.
Resolving deltas: 100% (841325/841325), done.
Checking out files: 100% (23692/23692), done.


In [12]:
%cd /content/tensorflow/
!sudo apt-get install openjdk-11-jdk
!echo "deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
!sudo apt-get install curl
!curl https://bazel.build/bazel-release.pub.gpg | sudo apt-key add -
!sudo apt-get update && sudo apt-get install bazel
!sudo apt-get update && sudo apt-get install bazel-3.1.0
#%cd /content/gdrive/My\ Drive/DeepLearning/tensorflow/


/content/tensorflow
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  openjdk-11-demo openjdk-11-source visualvm
The following NEW packages will be installed:
  openjdk-11-jdk
0 upgraded, 1 newly installed, 0 to remove and 11 not upgraded.
Need to get 2,268 kB of archives.
After this operation, 2,383 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 openjdk-11-jdk amd64 11.0.9+11-0ubuntu1~18.04.1 [2,268 kB]
Fetched 2,268 kB in 0s (17.7 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable

###Convert the Model to tflite with Bazel

In [13]:
%cd /content/tensorflow/
!sudo bazel run -c opt tensorflow/lite/toco:toco -- \
  --input_file=/content/gdrive/My\ Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/IG/tflite_graph.pb \
  --output_file=/content/gdrive/My\ Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/IG/detect.tflite \
  --input_shapes=1,300,300,3 \
  --input_arrays=normalized_input_image_tensor \
  --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3'  \
  --inference_type=QUANTIZED_UINT8 \
  --mean_values=128 \
  --std_values=128 \
  --change_concat_input_ranges=false \
  --allow_custom_ops

Streaming output truncated to the last 5000 lines.
[1,455 / 2,086] 2 actions running
    Compiling tensorflow/core/common_runtime/lower_if_op.cc; 2s local
[1,455 / 2,086] 2 actions running
    Compiling tensorflow/core/common_runtime/lower_if_op.cc; 3s local
[1,455 / 2,086] 2 actions running
    Compiling tensorflow/core/common_runtime/lower_if_op.cc; 4s local
[1,455 / 2,086] 2 actions running
    Compiling tensorflow/core/common_runtime/lower_if_op.cc; 5s local
[1,455 / 2,086] 2 actions running
    Compiling tensorflow/core/common_runtime/lower_if_op.cc; 6s local
[1,455 / 2,086] 2 actions running
    Compiling tensorflow/core/common_runtime/lower_if_op.cc; 7s local
[1,456 / 2,086] 2 actions running
    Compiling tensorflow/core/common_runtime/lower_while_op.cc; 7s local
[1,457 / 2,086] 2 actions, 1 running
    Compiling tensorflow/core/framework/log_memory.cc; 0s local
[1,457 / 2,086] 2 actions running
    Compiling tensorflow/core/framework/log_memory.cc; 1s local
[1,457 / 2,086] 2 a

###Finally compile the .tflite-graph for the Google Coral Edge TPU
from here: https://coral.withgoogle.com/docs/edgetpu/compiler/

In [1]:
#Download Packages
%cd /content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/IG
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get update
!sudo apt-get install edgetpu-compiler

[Errno 2] No such file or directory: '/content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/IG'
/content
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  20406      0 --:--:-- --:--:-- --:--:-- 19787
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease [6,332 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.3 kB]
Ign:7 https://developer.download.nvidia

In [4]:
#Compile
%cd /content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/IG
!edgetpu_compiler detect.tflite


/content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/IG
Edge TPU Compiler version 15.0.340273435

Model compiled successfully in 845 ms.

Input model: detect.tflite
Input size: 6.58MiB
Output model: detect_edgetpu.tflite
Output size: 7.02MiB
On-chip memory used for caching model parameters: 6.82MiB
On-chip memory remaining for caching model parameters: 861.75KiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 64
Operation log: detect_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 63
Number of operations that will run on CPU: 1
See the operation log file for individual oper

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

# New Section

you can now use the model detect_edgetpu.tflite with the edge tpu

# Let's run a (non edgeTPU) IG from Option A on test images!

## Imports

In [ ]:
import time
time.sleep(15*60)
%cd /content/gdrive/My\ Drive/DeepLearning/models/research/object_detection
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import glob2 #must be installed with pip install glob2
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# This is needed to display the images.
%matplotlib inline


## Model preparation
**Variables** Adapt number of classes to identify


In [ ]:
%cd /content/gdrive/My\ Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/

# Path to frozen detection graph. 
PATH_TO_FROZEN_GRAPH = './IG/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = './dataset/label.pbtxt'

#Number of classes
NUM_CLASSES = 2


## Load a (frozen) Tensorflow model into memory.

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [ ]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

## Detection

In [ ]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.9), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [ ]:
"""#Delete XML files out of folder
indic=[]
protc_files=glob2.glob('/content/gdrive/My Drive/FasterRCNN_TV9/Tensorflow_API-Custom_object_detection/dataset/testimages_/*')
print("found Annotations: \n", len(protc_files))
images_names=os.listdir('/content/gdrive/My Drive/FasterRCNN_TV9/Tensorflow_API-Custom_object_detection/dataset/testimages_/')
for i2 in range(len(protc_files)):
  if protc_files[i2].find("xml")>-1: #-1 means no sign found
    indic.append(i2)
if len(indic)>0:
  for i2 in range(len(indic)):
    os.remove(protc_files[indic[i2]])
    print('deleted file: ',protc_files[indic[i2]])
"""

## Choose which images to identify

In [ ]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
#TEST_IMAGE_PATHS=glob2.glob("/content/gdrive/My Drive/FasterRCNN_TV9/Tensorflow_API-Custom_object_detection/dataset/More_testimages/**.jpg")
#testimages_names=os.listdir('/content/gdrive/My Drive/FasterRCNN_TV9/Tensorflow_API-Custom_object_detection/dataset/More_testimages/')
TEST_IMAGE_PATHS=glob2.glob("/content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/dataset/testimages/*")
testimages_names=os.listdir('/content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/dataset/testimages/')
TEST_IMAGE_PATHS.sort()
testimages_names.sort()
print(TEST_IMAGE_PATHS)
import random
#import scipy.misc
from PIL import Image
shuffle_liste=list(range(0,(len(TEST_IMAGE_PATHS))))
#random.shuffle(shuffle_liste)
for i1 in range(0,len(TEST_IMAGE_PATHS)):
    #rand_var=random.randint(0,len(TEST_IMAGE_PATHS)-1)
    image = Image.open(TEST_IMAGE_PATHS[shuffle_liste[i1]])
    print(TEST_IMAGE_PATHS[shuffle_liste[i1]])
    #print('Hello')
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      max_boxes_to_draw=2,
      min_score_thresh=.3,
      line_thickness=3,
      skip_scores=False,
      skip_labels=False)
    #fig=plt.figure(figsize=IMAGE_SIZE)
    #plt.imshow(image_np)
    #plt.savefig('./Saved_detections/test%d.png'%(shuffle_liste[i1]))
    #plt.close()

    #scipy.misc.toimage(image_np, cmin=0.0, cmax=...).save('./Saved_detections/test%d.png'%(shuffle_liste[i1]))

    im = Image.fromarray(image_np)
    im.save('./dataset/Saved_detections/'+testimages_names[shuffle_liste[i1]])
    print(output_dict['detection_scores'][0])
#print(output_dict['detection_boxes'])
import time
time.sleep(20)

## Create a Video out of the stored data

In [ ]:
#Create a Video out of frames
%cd /content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection
import cv2
import glob2
#import numpy as np
#import glob
counter=0
img_array = []
image_list=glob2.glob('/content/gdrive/My Drive/DeepLearning/FasterRCNN_TV6/Tensorflow_API-Custom_object_detection/dataset/Saved_detections/*')
image_list.sort()
print(image_list[0],image_list[-1])
for filename in image_list:
	img = cv2.imread(filename)
	height, width, layers = img.shape
	size = (width,height)
	img_array.append(img)
	counter+=1
	print("read image ",counter)
print('part done, read images: '+str(counter))

out = cv2.VideoWriter('TrainingV1_RCNN_3.mp4',cv2.VideoWriter_fourcc(*'MP4V'), 25, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()